# Import

In [1]:
# This serves as a template which will guide you through the implementation of this task.  It is advised
# to first read the whole template and get a sense of the overall structure of the code before trying to fill in any of the TODO gaps
# First, we import necessary libraries:
import numpy as np
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset
import os
import torch
from torchvision import transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
from torchvision.models import resnet34, ResNet34_Weights
from sklearn.preprocessing import normalize
from google.colab import drive
drive.mount("/content/drive/")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

sample_dir = "/content/drive/MyDrive/IML/Projekt 3/dataset/Sample"
dataset_dir = "/content/drive/MyDrive/IML/Projekt 3/dataset/"
embeddings_dir = "/content/drive/MyDrive/IML/Projekt 3/dataset/embeddings.npy"
results_dir = "/content/drive/MyDrive/IML/Projekt 3/dataset/results.txt"
train_dir = "/content/drive/MyDrive/IML/Projekt 3/train_triplets.txt" 
test_dir = "/content/drive/MyDrive/IML/Projekt 3/test_triplets.txt"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# Generate Embeddings

In [ ]:
weights = ResNet34_Weights.DEFAULT
model = resnet34(weights=weights)
model.eval()

In [7]:
def generate_embeddings():
    """
    Transform, resize and normalize the images and then use a pretrained model to extract 
    the embeddings.
    """
    # Standardisieren hier die Daten auf z.B. selbe Dimensionen
    # Define a transform to pre-process the images; make pictures more similar 

    # 200x200 pixel transform transforms resize da manche zu gross/klein

    train_transforms = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()])

    train_dataset = datasets.ImageFolder(root=dataset_dir, transform=train_transforms)
    # Hint: adjust batch_size and num_workers to your PC configuration, so that you don't 
    # run out of memory
    train_loader = DataLoader(dataset=train_dataset,
                              batch_size=20,
                              shuffle=False,
                              pin_memory=True, num_workers=2) # num_workers = 2, batch_size = 20 
    print(train_loader)
    print(len(train_dataset))

    # Use ResNet50 as pretrained model to get embeddings
    # remove the last layers of the model to access the embeddings the model generates. 
    def copy_embeddings(module, input, output):

      """
      Copy embeddings from the penultimate layer.
      """

      output = output[:, :, 0, 0].detach().numpy().tolist()

      outputs.append(output)

    weights = ResNet34_Weights.DEFAULT
    model = resnet34(weights=weights)
    layer = model._modules.get('avgpool') # select desired layer
    _ = layer.register_forward_hook(copy_embeddings)
    embeddings = []
    embedding_size = 512 # Size of embeddings using ResNet18  
    model.eval() # activate the model

    # Generate image's embeddings for all images in train_loader and saves 
    # them in the list outputs

    outputs = []
    counter = 0
    for img, label in train_loader:
        print(counter)
        counter+=1
        _ = model(img)

    print(len(outputs)) # returns number of batches
    print(len(outputs[0])) # returns size of a batch
    print(len(outputs[7][9])) # returns embeddings_size

    num_images = len(train_dataset)
    embeddings = np.zeros((num_images, embedding_size))
    print(embeddings.shape)

    i = 0
    for batch in range(len(outputs)):
      print(i)
      for img in range(len(outputs[batch])):
        embeddings[i,:] = outputs[batch][img]
        i+=1
    
    np.save(embeddings_dir, embeddings)

In [8]:
generate_embeddings()

10201
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275


# Find number of workers

In [ ]:
import multiprocessing as mp
print(mp.cpu_count())

2


In [ ]:
from time import time
import multiprocessing as mp

for num_workers in range(2, mp.cpu_count()+1, 2):  
    train_loader = DataLoader(sample_dir,shuffle=True,num_workers=num_workers,batch_size=50,pin_memory=True)
    start = time()
    for epoch in range(1, 3):
        for i, data in enumerate(train_loader, 0):
          pass
        end = time()
        print("Finish with:{} second, num_workers={}".format(end - start, num_workers))

Finish with:0.04164481163024902 second, num_workers=2
Finish with:0.07740330696105957 second, num_workers=2


# Get Data

## Testing not relevant for main

In [2]:
    triplets = []
    with open(train_dir) as f:
        for line in f:
            triplets.append(line)

    # generate training data from triplets
    train_dataset = datasets.ImageFolder(dataset_dir,
                                         transform=None)
    filenames = [s[0].split('/')[-1].replace('.jpg', '') for s in train_dataset.samples]
    print(filenames)
    embeddings = np.load(embeddings_dir)
    print(embeddings)
    embeddings = normalize(embeddings)

KeyboardInterrupt: ignored

In [ ]:
print(embeddings.shape)

In [ ]:
print(len(filenames))

Für Train: Füge jedes Tripel 2 mal ein, als erste das Triple wo die ersten zwei ähnlich sind und füge eine 1 für die korrekte Ähnlichkeit im Geschmack ein und eine 0 wenn sie nicht ähnlich schmecken in y für den zweiten Fall; X enthält die Embeddings (2 mal; in der ersten Zeile die Ähnlichkeit; in der zweiten das andere

Falls nicht train: gib nur einmal Triple weiter (das mit 1) 

In [ ]:
    file_to_embedding = {}
    for i in range(len(filenames)):
        file_to_embedding[filenames[i]] = embeddings[i]
    X = []
    y = []
    counter = 0
    # use the individual embeddings to generate the features and labels for triplets
    for t in triplets:
        counter += 1
        if counter > 1:
          break
        emb = [file_to_embedding[a] for a in t.split()] # embedding triplets of pictures
        X.append(np.hstack([emb[0], emb[1], emb[2]])) # [emb[0], emb[1], emb[2]] merge the embeddings vertically
        # hstack merges them as a new column
        y.append(1)
        # Generating negative samples (data augmentation)
        X.append(np.hstack([emb[0], emb[2], emb[1]]))
        y.append(0)

In [ ]:
print(X)
print(y)

## Definition

In [3]:
def get_data(file, train=True):
    """
    Load the triplets from the file and generate the features and labels.

    input: file: string, the path to the file containing the triplets
          train: boolean, whether the data is for training or testing

    output: X: numpy array, the features, each column contains all embeddings of the pictures
            y: numpy array, the labels, 1 if the first picture tastes similar to the second picture
    """
    triplets = []
    with open(file) as f:
        for line in f:
            triplets.append(line)

    # generate training data from triplets
    train_dataset = datasets.ImageFolder(dataset_dir,
                                         transform=None)
    filenames = [s[0].split('/')[-1].replace('.jpg', '') for s in train_dataset.samples]
    embeddings = np.load(embeddings_dir)
    embeddings = normalize(embeddings) # Normalize the embeddings across the dataset
  

    file_to_embedding = {}
    for i in range(len(filenames)):
        file_to_embedding[filenames[i]] = embeddings[i]
    X = []
    y = []
    # use the individual embeddings to generate the features and labels for triplets
    for t in triplets:
        emb = [file_to_embedding[a] for a in t.split()]
        X.append(np.hstack([emb[0], emb[1], emb[2]]))
        y.append(1)
        # Generating negative samples (data augmentation)
        if train:
            X.append(np.hstack([emb[0], emb[2], emb[1]]))
            y.append(0)   
    X = np.vstack(X) # vertical stack
    y = np.hstack(y) # horizonal stack

    return X, y

# Create Loader From Np

In [4]:
# Hint: adjust batch_size and num_workers to your PC configuration, so that you don't run out of memory
def create_loader_from_np(X, y = None, train = True, batch_size=20, shuffle=True, num_workers = 2):
    """
    Create a torch.utils.data.DataLoader object from numpy arrays containing the data.

    input: X: numpy array, the features
           y: numpy array, the labels
    
    output: loader: torch.data.util.DataLoader, the object containing the data
    """
    if train:
        dataset = TensorDataset(torch.from_numpy(X).type(torch.float), 
                                torch.from_numpy(y).type(torch.long))
    else:
        dataset = TensorDataset(torch.from_numpy(X).type(torch.float))
    loader = DataLoader(dataset=dataset,
                        batch_size=batch_size,
                        shuffle=shuffle,
                        pin_memory=True, num_workers=num_workers)
    return loader

# Create Neural Network

In [13]:
# TODO: define a model. Here, the basic structure is defined, but you need to fill in the details
class Net(nn.Module):
    """
    The model class, which defines our classifier.
    """
    
    def __init__(self):
        
        # The constructor of the model.
        
        super().__init__()
        self.fc1 = nn.Linear(3*512,512)
        self.fc2 = nn.Linear(512,256)
        self.fc3 = nn.Linear(256,123)
        self.out = nn.Linear(123,1)

    def forward(self, x):
        """
        The forward pass of the model.

        input: x: torch.Tensor, the input to the model

        output: x: torch.Tensor, the output of the model
        """
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.out(x)
        return x

# Train Model

In [6]:
def train_model(train_loader):
    """
    The training procedure of the model; it accepts the training data, defines the model 
    and then trains it.

    input: train_loader: torch.data.util.DataLoader, the object containing the training data
    
    output: model: torch.nn.Module, the trained model
    """
    #os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
    model = Net()
    model.train()
    model.to(device)
    m = nn.Sigmoid()
    loss_function = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    
    losses = []
    n_epochs = 5 # 5 the best
    


    # TODO: define a loss function, optimizer and proceed with training. Hint: use the part 
    # of the training data as a validation split. After each epoch, compute the loss on the 
    # validation split and print it out. This enables you to see how your model is performing 
    # on the validation data before submitting the results on the server. After choosing the 
    # best model, train it on the whole training data.
    for epoch in range(n_epochs):    
        for batch_idx, (X, y) in enumerate(train_loader):
          optimizer.zero_grad()         
          output = model.forward(X)
          #print(output)
          output = torch.squeeze(output)
          #print(m(output))
          #print(y)
          loss = loss_function(m(output), y.float())
          losses.append(loss) 
          loss.backward()
          optimizer.step()
          if batch_idx % 100 == 0:
            print('Epoch {}, Batch idx {}, loss {}'.format(
                   epoch, batch_idx, loss.item()))
                  
    return model

# Test Model

In [7]:
def test_model(model, loader):
    """
    The testing procedure of the model; it accepts the testing data and the trained model and 
    then tests the model on it.

    input: model: torch.nn.Module, the trained model
           loader: torch.data.util.DataLoader, the object containing the testing data
        
    output: None, the function saves the predictions to a results.txt file
    """
    model.eval()
    predictions = []
    # Iterate over the test data
    with torch.no_grad(): # We don't need to compute gradients for testing
        for [x_batch] in loader:
            x_batch= x_batch.to(device)
            predicted = model(x_batch)
            predicted = predicted.cpu().numpy()
            # Rounding the predictions to 0 or 1
            predicted[predicted >= 0.5] = 1
            predicted[predicted < 0.5] = 0
            predictions.append(predicted)
        predictions = np.vstack(predictions)
    np.savetxt(results_dir, predictions, fmt='%i')

# Main Function

In [8]:
# Main function. You don't have to change this
if __name__ == '__main__':
    TRAIN_TRIPLETS = train_dir
    TEST_TRIPLETS = test_dir

    # generate embedding for each image in the dataset
    if(os.path.exists(embeddings_dir) == False):
        generate_embeddings()

In [9]:
    # load the training and testing data 
    X, y = get_data(TRAIN_TRIPLETS)
    X_test, _ = get_data(TEST_TRIPLETS, train=False)

In [10]:
    # Create data loaders for the training and testing data
    train_loader = create_loader_from_np(X, y, train = True, batch_size=20)
    test_loader = create_loader_from_np(X_test, train = False, batch_size=20, shuffle=False)

In [14]:
    # define a model and train it
    model = train_model(train_loader)

Epoch 0, Batch idx 0, loss 0.6900358200073242
Epoch 0, Batch idx 100, loss 0.6986687779426575
Epoch 0, Batch idx 200, loss 0.695382297039032
Epoch 0, Batch idx 300, loss 0.6959062814712524
Epoch 0, Batch idx 400, loss 0.693827211856842
Epoch 0, Batch idx 500, loss 0.689926028251648
Epoch 0, Batch idx 600, loss 0.6855530738830566
Epoch 0, Batch idx 700, loss 0.7022871971130371
Epoch 0, Batch idx 800, loss 0.6834679841995239
Epoch 0, Batch idx 900, loss 0.6923227310180664
Epoch 0, Batch idx 1000, loss 0.6856356859207153
Epoch 0, Batch idx 1100, loss 0.6926175951957703
Epoch 0, Batch idx 1200, loss 0.6874204874038696
Epoch 0, Batch idx 1300, loss 0.7272890210151672
Epoch 0, Batch idx 1400, loss 0.723486065864563
Epoch 0, Batch idx 1500, loss 0.7706869840621948
Epoch 0, Batch idx 1600, loss 0.6575467586517334
Epoch 0, Batch idx 1700, loss 0.5840273499488831
Epoch 0, Batch idx 1800, loss 0.6673463582992554
Epoch 0, Batch idx 1900, loss 0.5777016878128052
Epoch 0, Batch idx 2000, loss 0.5786

In [ ]:
    # test the model on the test data
    test_model(model, test_loader)
    print("Results saved to results.txt")

## Test - nor relevant for main

In [ ]:
    TRAIN_TRIPLETS = train_dir
    TEST_TRIPLETS = test_dir

    # generate embedding for each image in the dataset
    if(os.path.exists(embeddings_dir) == False):
        print("generate embeddings")
        generate_embeddings()
        # load the training and testing data 

In [ ]:
X, y = get_data(TRAIN_TRIPLETS)
X_test, _ = get_data(TEST_TRIPLETS, train=False)

In [ ]:
print(len(X[0]))
print(len(X))
print(len(y))

1536
119030
119030


In [ ]:
train_loader = create_loader_from_np(X, y, train = True, batch_size=20)
test_loader = create_loader_from_np(X_test, train = False, batch_size=20, shuffle=False)